# Задание 3
Подберите несколько датасетов с большим количеством предикторов (>20) из UCI репозитория или любого другого (классификация и/или регрессия)). Реализуйте gaussian process (можно взять готовый, но скорее всего его все равно придется модифицировать), для подбора гиперпараметров random forest (или какого-либо бустинга) (смотрите лекцию 3). Сравните скорость с random search (попытайтесь превзойти).

In [25]:
from io import StringIO, BytesIO
import requests
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

In [26]:
def load_spambase():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
    response = requests.get(url,
                            verify=False)  # без этого все падало с ошибкой certificate verify failed: certificate has expired, пришлось обходить ошибку игнорированием проверки сертификата
    spambase = pd.read_csv(StringIO(response.text), header=None, compression='zip')
    X = spambase.iloc[:, :-1]
    y = spambase.iloc[:, -1]
    return X, y


def load_arrhythmia():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data"
    response = requests.get(url, verify=False)
    arrhythmia = pd.read_csv(StringIO(response.text), header=None, compression='zip', na_values="?")
    arrhythmia.fillna(arrhythmia.mean(), inplace=True)  # заполняем пропущенные значения
    X = arrhythmia.iloc[:, :-1]
    y = arrhythmia.iloc[:, -1]
    return X, y


def load_breast_cancer_wisconsin():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    response = requests.get(url, verify=False)
    df = pd.read_csv(StringIO(response.text), header=None)
    X = df.iloc[:, 2:]
    y = df.iloc[:, 1]
    return X, y

In [27]:
datasets = {
    "Spambase": load_spambase,
    "Arrhythmia": load_arrhythmia,
    "Breast Cancer Wisconsin": load_breast_cancer_wisconsin
}

results = {}

In [28]:
from time import time

for name, load_function in datasets.items():
    print(f"Processing dataset: {name}")

    # загружаем данные и измеряем время
    start_time = time()
    X, y = load_function()
    load_time = time() - start_time

    # нормализация данных
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # делим на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # для оптимизации
    def evaluate_model(params):
        n_estimators, max_depth, min_samples_leaf = params
        model = RandomForestClassifier(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            min_samples_leaf=int(min_samples_leaf),
            random_state=42
        )
        return -np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy'))


    # границы гиперпараметров
    bounds = [
        (10, 200),  # n_estimators
        (2, 20),  # max_depth
        (1, 10)  # min_samples_leaf
    ]

    # оптимизация
    start_time = time()
    result = minimize(
        evaluate_model,
        x0=[100, 10, 5],  # начальные значения гиперпараметров
        bounds=bounds,
        method='L-BFGS-B'
    )
    minimize_time = time() - start_time

    best_params = result.x
    best_score = -result.fun

    # сравнение с Random Search
    from sklearn.model_selection import ParameterSampler

    param_grid = {
        'n_estimators': range(10, 200),
        'max_depth': range(2, 20),
        'min_samples_leaf': range(1, 10)
    }
    random_search = ParameterSampler(param_grid, n_iter=50, random_state=42)

    start_time = time()
    best_random_score = float('-inf')
    for params in random_search:
        score = evaluate_model([
            params['n_estimators'],
            params['max_depth'],
            params['min_samples_leaf']
        ])
        if score > best_random_score:
            best_random_score = score
    random_search_time = time() - start_time

    results[name] = {
        "Load Time": load_time,
        "Best Parameters (Minimize)": best_params,
        "Best Score (Minimize)": best_score,
        "Minimize Time": minimize_time,
        "Best Random Search Score": abs(best_random_score),
        "Random Search Time": random_search_time
    }

Processing dataset: Spambase


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/var/folders/6t/0bc7rzr91pd3316c59fnhvwc0000gn/T/ipykernel_36415/1383866971.py:4: RuntimeWarning: compression has no effect when passing a non-binary object as input.
  spambase = pd.read_csv(StringIO(response.text), header=None, compression='zip')


Processing dataset: Arrhythmia


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/var/folders/6t/0bc7rzr91pd3316c59fnhvwc0000gn/T/ipykernel_36415/1383866971.py:12: RuntimeWarning: compression has no effect when passing a non-binary object as input.
  arrhythmia = pd.read_csv(StringIO(response.text), header=None, compression='zip', na_values="?")
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  war

Processing dataset: Breast Cancer Wisconsin


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [29]:
for name, result in results.items():
    print(f"Dataset: {name}")
    for key, value in result.items():
        print(f"  {key}: {value}")
    print()

Dataset: Spambase
  Load Time: 1.618600845336914
  Best Parameters (Minimize): [100.  10.   5.]
  Best Score (Minimize): 0.9366869152603666
  Minimize Time: 2.0346179008483887
  Best Random Search Score: 0.879621689439133
  Random Search Time: 28.48149299621582

Dataset: Arrhythmia
  Load Time: 1.397935152053833
  Best Parameters (Minimize): [100.  10.   5.]
  Best Score (Minimize): 0.6536730945821855
  Minimize Time: 0.832345724105835
  Best Random Search Score: 0.5678833792470156
  Random Search Time: 12.255026817321777

Dataset: Breast Cancer Wisconsin
  Load Time: 1.0439159870147705
  Best Parameters (Minimize): [100.  10.   5.]
  Best Score (Minimize): 0.9450447310328802
  Minimize Time: 0.6732957363128662
  Best Random Search Score: 0.9318577901707913
  Random Search Time: 9.94124984741211



In [24]:
from io import StringIO, BytesIO
import requests
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import time


def load_spambase():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
    response = requests.get(url, verify=False)
    spambase = pd.read_csv(StringIO(response.text), header=None)
    X = spambase.iloc[:, :-1]
    y = spambase.iloc[:, -1]
    return X, y


def load_arrhythmia():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data"
    response = requests.get(url, verify=False)
    arrhythmia = pd.read_csv(StringIO(response.text), header=None, na_values="?")
    arrhythmia.fillna(arrhythmia.mean(), inplace=True)  # Заполняем пропущенные значения
    X = arrhythmia.iloc[:, :-1]
    y = arrhythmia.iloc[:, -1]
    return X, y


def load_breast_cancer_wisconsin():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    response = requests.get(url, verify=False)
    df = pd.read_csv(StringIO(response.text), header=None)
    X = df.iloc[:, 2:]
    y = df.iloc[:, 1]
    return X, y


# Список функций загрузки датасетов
datasets = {
    "Spambase": load_spambase,
    "Arrhythmia": load_arrhythmia,
    "Breast Cancer Wisconsin": load_breast_cancer_wisconsin
}

results = {}

for name, load_function in datasets.items():
    print(f"Processing dataset: {name}")

    # Загружаем данные и измеряем время
    start_time = time.time()
    X, y = load_function()
    load_time = time.time() - start_time

    # Нормализация данных
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # Определяем функцию для оптимизации
    def evaluate_model(params):
        n_estimators, max_depth, min_samples_leaf = params
        model = RandomForestClassifier(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            min_samples_leaf=int(min_samples_leaf),
            random_state=42
        )
        return -np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy'))


    # Границы гиперпараметров
    bounds = [
        (10, 200),  # n_estimators
        (2, 20),  # max_depth
        (1, 10)  # min_samples_leaf
    ]

    # Оптимизация с использованием scipy.optimize.minimize
    start_time = time.time()
    result = minimize(
        evaluate_model,
        x0=[100, 10, 5],  # Начальные значения гиперпараметров
        bounds=bounds,
        method='L-BFGS-B'
    )
    minimize_time = time.time() - start_time

    best_params = result.x
    best_score = -result.fun

    # Сравнение с Random Search
    from sklearn.model_selection import ParameterSampler

    param_grid = {
        'n_estimators': range(10, 200),
        'max_depth': range(2, 20),
        'min_samples_leaf': range(1, 10)
    }
    random_search = ParameterSampler(param_grid, n_iter=50, random_state=42)

    start_time = time.time()
    best_random_score = float('-inf')
    for params in random_search:
        score = evaluate_model([
            params['n_estimators'],
            params['max_depth'],
            params['min_samples_leaf']
        ])
        if score > best_random_score:
            best_random_score = score
    random_search_time = time.time() - start_time

    # Сохраняем результаты
    results[name] = {
        "Load Time": load_time,
        "Best Parameters (Minimize)": best_params,
        "Best Score (Minimize)": best_score,
        "Minimize Time": minimize_time,
        "Best Random Search Score": abs(best_random_score),
        "Random Search Time": random_search_time
    }

# Вывод результатов
for name, result in results.items():
    print(f"Dataset: {name}")
    for key, value in result.items():
        print(f"  {key}: {value}")
    print()


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Best Parameters (Minimize): [100.  10.   5.]
Best Score (Minimize): 0.9450447310328802
Best Random Search Score: 0.9318577901707913


In [31]:
from io import StringIO, BytesIO
import requests
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterSampler
from skopt import gp_minimize
from skopt.space import Integer, Real
import time

In [ ]:
# грузим датасеты
def load_spambase():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
    response = requests.get(url, verify=False)
    spambase = pd.read_csv(StringIO(response.text), header=None)
    X = spambase.iloc[:, :-1]
    y = spambase.iloc[:, -1]
    return X, y


def load_arrhythmia():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data"
    response = requests.get(url, verify=False)
    arrhythmia = pd.read_csv(StringIO(response.text), header=None, na_values="?")
    arrhythmia.fillna(arrhythmia.mean(), inplace=True)
    X = arrhythmia.iloc[:, :-1]
    y = arrhythmia.iloc[:, -1]
    return X, y


def load_breast_cancer_wisconsin():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    response = requests.get(url, verify=False)
    df = pd.read_csv(StringIO(response.text), header=None)
    X = df.iloc[:, 2:]
    y = df.iloc[:, 1]
    return X, y

In [35]:
datasets = {
    "Spambase": load_spambase,
    "Arrhythmia": load_arrhythmia,
    "Breast Cancer Wisconsin": load_breast_cancer_wisconsin
}

results = []

In [36]:
for name, load_function in datasets.items():
    print(f"Processing dataset: {name}")

    # грузим + измеряем время
    start_time = time.time()
    X, y = load_function()

    # нормализуем
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # делим данные на выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    # это функция для оптимизации
    def evaluate_model(params):
        n_estimators, max_depth, min_samples_leaf = params
        model = RandomForestClassifier(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            min_samples_leaf=int(min_samples_leaf),
            random_state=42
        )
        return -np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy'))


    # гиперпараметры
    space = [
        Integer(10, 200, name='n_estimators'),
        Integer(2, 20, name='max_depth'),
        Integer(1, 10, name='min_samples_leaf')
    ]

    # оптимизация
    result_gp = gp_minimize(
        evaluate_model,
        dimensions=space,
        n_calls=50,  # итерации
        random_state=42
    )
    gp_time = time.time() - start_time

    best_gp_params = result_gp.x
    best_gp_score = -result_gp.fun

    # Random Search
    param_grid = {
        'n_estimators': range(10, 200),
        'max_depth': range(2, 20),
        'min_samples_leaf': range(1, 10)
    }
    random_search = ParameterSampler(param_grid, n_iter=50, random_state=42)

    start_time = time.time()
    best_random_score = float('-inf')  # тк у нас в функции возвращается отрицательное значение
    for params in random_search:
        score = evaluate_model([
            params['n_estimators'],
            params['max_depth'],
            params['min_samples_leaf']
        ])
        if score > best_random_score:
            best_random_score = score
    random_search_time = time.time() - start_time

    # сохраняем результаты
    results.append({
        "Dataset": name,
        "Best GP Parameters": best_gp_params,
        "Best GP Score": best_gp_score,
        "GP Time": gp_time,
        "Best Random Search Score": abs(best_random_score),
        "Random Search Time": random_search_time
    })

Processing dataset: Spambase


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Processing dataset: Arrhythmia


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-p

Processing dataset: Breast Cancer Wisconsin


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive.ics.uci.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [37]:
results_df = pd.DataFrame(results)
print(results_df)

                   Dataset Best GP Parameters  Best GP Score    GP Time  \
0                 Spambase       [200, 19, 1]       0.947829  62.885500   
1               Arrhythmia       [200, 18, 1]       0.728558  27.804846   
2  Breast Cancer Wisconsin        [30, 13, 1]       0.958217  19.339130   

   Best Random Search Score  Random Search Time  
0                  0.879622           29.214304  
1                  0.567883           12.374886  
2                  0.931858            9.998734  


Как видно, GP выдает результаты лучше, чем RandomSearch, зато работает на каждом датасете в среднем в 2 раза дольше. То есть GP более эффективен для оптимизации гиперпараметров. На более маленьких датасетах (например Breast Cancer Wisconsin) результаты GP и RandomSearch почти одинаковые, а вот на более больших датасетах разница становится уже существенной.
Актуально использовать GP, когда требуется высокая точность и есть вычислительные мощности, RandomSearch подходит, если данных не слишком много + если требуется высокая скорость

Дополнительно: поэкспериментируйте с критериями подбора следующей точки (лекция 3) и ядрами, а также с самими критериями качества (есть ли отличие в эффективности gaussian process для разных критериев)

In [38]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer
from skopt.learning.gaussian_process.kernels import RBF, Matern, DotProduct
import seaborn as sns
import matplotlib.pyplot as plt

In [50]:
# грузим датасеты
def load_spambase():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
    response = requests.get(url, verify=False)
    spambase = pd.read_csv(StringIO(response.text), header=None)
    X = spambase.iloc[:, :-1]
    y = spambase.iloc[:, -1]
    y = y.dropna()  # Удаляем строки с NaN в целевой переменной
    X = X.loc[y.index]  # Удаляем соответствующие строки в X
    return X, y


def load_arrhythmia():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data"
    response = requests.get(url, verify=False)
    arrhythmia = pd.read_csv(StringIO(response.text), header=None, na_values="?")
    arrhythmia.fillna(arrhythmia.mean(), inplace=True)
    X = arrhythmia.iloc[:, :-1]
    y = arrhythmia.iloc[:, -1]
    y = y.dropna()  # Удаляем строки с NaN в целевой переменной
    X = X.loc[y.index]  # Удаляем соответствующие строки в X
    return X, y


def load_breast_cancer_wisconsin():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    response = requests.get(url, verify=False)
    df = pd.read_csv(StringIO(response.text), header=None)
    X = df.iloc[:, 2:]
    y = df.iloc[:, 1]
    y = y.dropna()  # Удаляем строки с NaN в целевой переменной
    X = X.loc[y.index]  # Удаляем соответствующие строки в X
    return X, y

In [51]:
from sklearn.gaussian_process import GaussianProcessRegressor


# Функция для экспериментов с GPO
# она будет запускать оптимизацию GP с разными параметрами: разные критерии подбора след точки (criterion)
# разные kernel, а также смотрим разные метрики
def experiment_with_gp(X_train, y_train, criterion, kernel, scoring_metric):
    # Определяем функцию оценки модели
    def evaluate_model(params):
        n_estimators, max_depth, min_samples_leaf = params
        model = RandomForestClassifier(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            min_samples_leaf=int(min_samples_leaf),
            random_state=42
        )
        scorer = make_scorer(scoring_metric, needs_proba=True if scoring_metric == roc_auc_score else False)
        return -np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring=scorer))

    # пространство гиперпараметров
    space = [
        Integer(10, 200, name='n_estimators'),
        Integer(2, 20, name='max_depth'),
        Integer(1, 10, name='min_samples_leaf')
    ]
    gp = GaussianProcessRegressor(kernel=kernel, random_state=42)
    # запускаем  Gaussian Process Optimization
    start_time = time.time()
    result_gp = gp_minimize(
        evaluate_model,
        dimensions=space,
        acq_func=criterion,  # 'EI', 'PI', 'LCB'
        base_estimator=None,
        n_calls=50,
        random_state=42
    )
    gp_time = time.time() - start_time

    return {
        "Best GP Parameters": result_gp.x,
        "Best GP Score": -result_gp.fun,
        "GP Time": gp_time,
        "Criterion": criterion,
        "Kernel": str(kernel),
        "Scoring Metric": scoring_metric.__name__
    }

In [52]:
# Список функций загрузки датасетов
datasets = {
    "Spambase": load_spambase,
    "Arrhythmia": load_arrhythmia,
    "Breast Cancer Wisconsin": load_breast_cancer_wisconsin
}

# Список критериев подбора точки
c = ['EI', 'PI', 'LCB']

# Список ядер
kernels = [RBF(), Matern(), DotProduct()]

# Метрики качества
metrics = [accuracy_score, f1_score, roc_auc_score]

# Сохраняем результаты
results = []

In [54]:
import warnings

warnings.filterwarnings("ignore")  # чтобы не захламлять вывод
for name, load_function in datasets.items():
    print(f"Processing dataset: {name}")
    X, y = load_function()
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for criterion in c:
        for kernel in kernels:
            for metric in metrics:
                result = experiment_with_gp(X_train, y_train, criterion, kernel, metric)
                result["Dataset"] = name
                results.append(result)

Processing dataset: Spambase
Processing dataset: Arrhythmia


ValueError: Input y contains NaN.

Сначала выведу результаты для Spambase, потом отдельно посмотрю 2 оставшихся датасета, хотя вообще я везде добавила удаление NaN значений

In [58]:
# выводим результаты
results_df = pd.DataFrame(results)
print(results_df)

   Best GP Parameters  Best GP Score    GP Time Criterion  \
0        [196, 19, 1]       0.948644  39.790772        EI   
1        [200, 19, 1]       0.931473  42.148857        EI   
2        [111, 19, 1]       0.984896  34.750708        EI   
3        [196, 19, 1]       0.948644  40.256498        EI   
4        [200, 19, 1]       0.931473  42.491195        EI   
5        [111, 19, 1]       0.984896  35.136075        EI   
6        [196, 19, 1]       0.948644  40.489586        EI   
7        [200, 19, 1]       0.931473  42.433335        EI   
8        [111, 19, 1]       0.984896  35.258723        EI   
9        [196, 19, 1]       0.948644  41.863796        PI   
10       [200, 19, 1]       0.931473  37.986253        PI   
11       [111, 19, 1]       0.984896  35.000472        PI   
12       [196, 19, 1]       0.948644  41.859312        PI   
13       [200, 19, 1]       0.931473  38.201834        PI   
14       [111, 19, 1]       0.984896  35.017424        PI   
15       [196, 19, 1]   

In [87]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
def load_arrhythmia():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/arrhythmia/arrhythmia.data"
    response = requests.get(url, verify=False)
    arrhythmia = pd.read_csv(StringIO(response.text), header=None, na_values="?")
    # Последний столбец - целевая переменная
    target_col = arrhythmia.columns[-1]
    # Удаляем строки без значения целевого столбца
    arrhythmia = arrhythmia.dropna(subset=[target_col])
    # Преобразуем целевой столбец в int
    arrhythmia[target_col] = pd.to_numeric(arrhythmia[target_col], errors='coerce')
    arrhythmia = arrhythmia.dropna(subset=[target_col])
    arrhythmia[target_col] = arrhythmia[target_col].astype(int)
    X = arrhythmia.iloc[:, :-1]
    y = arrhythmia.iloc[:, -1]
    # Удаляем столбцы, полностью состоящие из NaN
    X = X.dropna(axis=1, how='all')
    # Заполняем пропуски в X средними значениями
    X = X.fillna(X.mean())

    # Проверяем ещё раз наличие NaN
    if y.isna().any():
        print("Warning: y still has NaN after cleaning!")
    if X.isna().sum().sum() > 0:
        print("Warning: X still has NaN after cleaning!")
    return X, y

In [88]:
from sklearn.model_selection import StratifiedKFold

# как и выше, только пришлось чуть переделать
def experiment_with_gp(X_train, y_train, criterion, kernel, scoring_metric):
    # проверки нужны, чтобы все ок работало, если многоклассовая классификация
    if scoring_metric == roc_auc_score:
        scorer = make_scorer(scoring_metric, needs_proba=True, multi_class='ovr')
    elif scoring_metric == f1_score:
        scorer = make_scorer(scoring_metric, average='macro')
    else:
        scorer = make_scorer(scoring_metric)

    # Возьму стратифицированное разбиение, чтобы минимизировать риск пропадания классов
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    def evaluate_model(params):
        n_estimators, max_depth, min_samples_leaf = params
        model = RandomForestClassifier(
            n_estimators=int(n_estimators),
            max_depth=int(max_depth),
            min_samples_leaf=int(min_samples_leaf),
            random_state=42
        )
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scorer)
        return -np.mean(cv_scores)

    # пространство гиперпараметров
    space = [
        Integer(10, 200, name='n_estimators'),
        Integer(2, 20, name='max_depth'),
        Integer(1, 10, name='min_samples_leaf')
    ]
    gp = GaussianProcessRegressor(kernel=kernel, random_state=42)

    start_time = time.time()
    result_gp = gp_minimize(
        evaluate_model,
        dimensions=space,
        acq_func=criterion,  # 'EI', 'PI', 'LCB'
        base_estimator=None,
        n_calls=50,
        random_state=42
    )
    gp_time = time.time() - start_time

    return {
        "Best GP Parameters": result_gp.x,
        "Best GP Score": -result_gp.fun,
        "GP Time": gp_time,
        "Criterion": criterion,
        "Kernel": str(kernel),
        "Scoring Metric": scoring_metric.__name__
    }


In [91]:
import warnings

warnings.filterwarnings("ignore")  # чтобы не захламлять вывод
print(f"Processing dataset: Arrhythmia")
X, y = load_arrhythmia()
print("NaNs in y:", y.isna().sum())  # надо было проверить, что нет NaN из-за неуходившей ValueError

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

c = ['EI', 'PI', 'LCB']
kernels = [RBF(), Matern(), DotProduct()]
metrics = [accuracy_score, f1_score]
results = []
for criterion in c:
    for kernel in kernels:
        for metric in metrics:
            result = experiment_with_gp(X_train, y_train, criterion, kernel, metric)
            result["Dataset"] = "Arrhythmia"
            results.append(result)

Processing dataset: Arrhythmia
NaNs in y: 0


У меня все падало с ValueError: input y contains NaN, я почистила все данные, это не привело к успеху, а потом я поняла, что дело не в том, как я загружаю датасет, а в том, что roc_auc_score возвращал NaN, потому что в некоторых фолдах кросс-валидации не было примеров для одного или нескольких классов, то есть вычисление ROC-кривой было нереально сделать. Это возникало из-за неравномерного распределения классов
Убрала этот скор, все работает ок

In [92]:
# выводим результаты
results_df = pd.DataFrame(results)
print(results_df)

   Best GP Parameters  Best GP Score    GP Time Criterion  \
0        [200, 20, 1]       0.711869  19.146470        EI   
1         [25, 18, 1]       0.393743  17.948938        EI   
2        [200, 20, 1]       0.711869  18.434840        EI   
3         [25, 18, 1]       0.393743  17.906764        EI   
4        [200, 20, 1]       0.711869  18.582603        EI   
5         [25, 18, 1]       0.393743  17.802387        EI   
6         [71, 18, 1]       0.722957  19.550469        PI   
7         [71, 20, 1]       0.385232  19.820757        PI   
8         [71, 18, 1]       0.722957  19.897916        PI   
9         [71, 20, 1]       0.385232  20.386963        PI   
10        [71, 18, 1]       0.722957  20.441300        PI   
11        [71, 20, 1]       0.385232  20.041394        PI   
12       [200, 20, 1]       0.711869  20.759391       LCB   
13       [200, 14, 1]       0.387591  19.943882       LCB   
14       [200, 20, 1]       0.711869  21.052318       LCB   
15       [200, 14, 1]   

In [100]:
def load_breast_cancer_wisconsin():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
    response = requests.get(url, verify=False)
    df = pd.read_csv(StringIO(response.text), header=None)
    X = df.iloc[:, 2:]
    y = df.iloc[:, 1]
    y = y.dropna()  # Удаляем строки с NaN в целевой переменной
    X = X.loc[y.index]  # Удаляем соответствующие строки в X
    return X, y

In [103]:
import warnings

warnings.filterwarnings("ignore")  # чтобы не захламлять вывод
print(f"Processing dataset: Breast Cancer Wisconsin")
X, y = load_breast_cancer_wisconsin()
print("NaNs in y:", y.isna().sum())  # надо было проверить, что нет NaN из-за неуходившей ValueError

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

c = ['EI', 'PI', 'LCB']
kernels = [RBF(), Matern(), DotProduct()]
metrics = [accuracy_score]
results = []
for criterion in c:
    for kernel in kernels:
        for metric in metrics:
            result = experiment_with_gp(X_train, y_train, criterion, kernel, metric)
            result["Dataset"] = "Breast Cancer Wisconsin"
            results.append(result)

Processing dataset: Breast Cancer Wisconsin
NaNs in y: 0


In [104]:
# выводим результаты
results_df = pd.DataFrame(results)
print(results_df)

  Best GP Parameters  Best GP Score    GP Time Criterion  \
0       [127, 13, 1]       0.962632  16.980731        EI   
1       [127, 13, 1]       0.962632  16.895294        EI   
2       [127, 13, 1]       0.962632  17.109814        EI   
3        [16, 20, 1]       0.964840  17.136276        PI   
4        [16, 20, 1]       0.964840  17.258928        PI   
5        [16, 20, 1]       0.964840  16.830273        PI   
6       [127, 13, 1]       0.962632  15.960571       LCB   
7       [127, 13, 1]       0.962632  17.184243       LCB   
8       [127, 13, 1]       0.962632  16.883649       LCB   

                           Kernel  Scoring Metric                  Dataset  
0             RBF(length_scale=1)  accuracy_score  Breast Cancer Wisconsin  
1  Matern(length_scale=1, nu=1.5)  accuracy_score  Breast Cancer Wisconsin  
2           DotProduct(sigma_0=1)  accuracy_score  Breast Cancer Wisconsin  
3             RBF(length_scale=1)  accuracy_score  Breast Cancer Wisconsin  
4  Matern(leng

Для этого датасета с f1_score тоже была беда (те же причины что и у roc_auc_score), и тк тут не числовые данные, то трюк с преобразованием в нумерик не сработал, поэтому, чтобы избежать той же ValueError пришлось пожертвовать и f1_score тоже
А другие датасеты - беда, потому что в UCI очень кривая файловая система, не всегда очевидно, как конструировать ссылки на сами данные, были еще некоторые варианты, но там либо все падало из-за просроченного сертификата (не получалось прочитать данные), либо просто была ошибка при соединении